# Анализ цен на авиабилеты LED → SVX

Мониторинг цен на маршруте **Санкт-Петербург → Екатеринбург**.

**Источник данных:**
- `public_dm.dm_price_by_days_before_departure` — витрина "за сколько дней покупать"
- `public_dm.dm_price_by_day_and_hour` — витрина "в какое время искать"

In [ ]:
import pandas as pd
import plotly.express as px
from sqlalchemy import create_engine
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.0f}'.format)

engine = create_engine(
    f"postgresql://{os.getenv('POSTGRES_USER', 'airflow')}:{os.getenv('POSTGRES_PASSWORD', 'airflow')}@{os.getenv('POSTGRES_HOST', 'localhost')}:5432/{os.getenv('POSTGRES_DB', 'aviasales')}"
)
print("Подключено к БД")

---
## 1. Загрузка данных

Загружаем обе витрины целиком. Агрегация выполняется в pandas.

In [ ]:
# Загрузка витрин (по одному запросу на витрину)
df_days = pd.read_sql(
    "SELECT * FROM public_dm.dm_price_by_days_before_departure ORDER BY days_until_departure",
    engine
)

df_time = pd.read_sql(
    "SELECT * FROM public_dm.dm_price_by_day_and_hour ORDER BY fetch_day_of_week, fetch_hour",
    engine
)

print(f"Витрина 'дни до вылета': {len(df_days)} строк")
print(f"Витрина 'день+час': {len(df_time)} строк")

---
## 2. За сколько дней до вылета покупать?

**Витрина:** `dm_price_by_days_before_departure`

**Методология:** Для каждого рейса считаем отклонение цены от средней цены этого же рейса. Отрицательное значение = цены ниже средней.

In [ ]:
display(df_days.head(20))

In [ ]:
fig = px.bar(
    df_days,
    x='days_until_departure',
    y='avg_price_deviation',
    title='Отклонение цены от средней по дням до вылета',
    labels={'days_until_departure': 'Дней до вылета', 'avg_price_deviation': 'Отклонение (руб)'},
    color='avg_price_deviation',
    color_continuous_scale='RdYlGn_r'
)
fig.add_hline(y=0, line_dash="dash", line_color="gray")
fig.update_layout(template='plotly_white')
fig.show()

best_day = df_days.loc[df_days['avg_price_deviation'].idxmin()]
print(f"\nЛучший период: за {int(best_day['days_until_departure'])} дней до вылета")
print(f"Отклонение: {int(best_day['avg_price_deviation'])} руб от средней")

---
## 3. В какое время искать билеты?

**Витрина:** `dm_price_by_day_and_hour`

**Методология:** Для каждого рейса считаем отклонение цены от средней цены этого же рейса. Группируем по времени парсинга.

### 3.1 По дням недели

In [ ]:
# Агрегация по дню недели в pandas
df_by_day = df_time.groupby(['fetch_day_of_week', 'day_name']).agg(
    avg_deviation=('avg_price_deviation', 'mean'),
    avg_price=('avg_price', 'mean'),
    observations=('observations_count', 'sum')
).round(0).reset_index().sort_values('fetch_day_of_week')

display(df_by_day)

fig = px.bar(
    df_by_day,
    x='day_name',
    y='avg_deviation',
    title='Отклонение цены по дням недели',
    labels={'day_name': 'День недели', 'avg_deviation': 'Отклонение (руб)'},
    color='avg_deviation',
    color_continuous_scale='RdYlGn_r'
)
fig.add_hline(y=0, line_dash="dash", line_color="gray")
fig.update_layout(template='plotly_white')
fig.show()

### 3.2 По часам

In [ ]:
# Агрегация по часу в pandas
df_by_hour = df_time.groupby('fetch_hour').agg(
    avg_deviation=('avg_price_deviation', 'mean'),
    avg_price=('avg_price', 'mean'),
    observations=('observations_count', 'sum')
).round(0).reset_index()

fig = px.bar(
    df_by_hour,
    x='fetch_hour',
    y='avg_deviation',
    title='Отклонение цены по часам',
    labels={'fetch_hour': 'Час', 'avg_deviation': 'Отклонение (руб)'},
    color='avg_deviation',
    color_continuous_scale='RdYlGn_r'
)
fig.add_hline(y=0, line_dash="dash", line_color="gray")
fig.update_layout(template='plotly_white')
fig.show()

best_hour = df_by_hour.loc[df_by_hour['avg_deviation'].idxmin()]
print(f"Лучший час: {int(best_hour['fetch_hour'])}:00")
print(f"Отклонение: {int(best_hour['avg_deviation'])} руб от средней")

### 3.3 Детально: день + час (heatmap)

In [ ]:
# Pivot для heatmap (данные уже загружены в df_time)
pivot = df_time.pivot(index='day_name', columns='fetch_hour', values='avg_price_deviation')

# Сортировка дней недели
day_order = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 'Воскресенье']
pivot = pivot.reindex([d for d in day_order if d in pivot.index])

fig = px.imshow(
    pivot,
    title='Отклонение цены: день недели × час',
    labels={'x': 'Час', 'y': 'День недели', 'color': 'Отклонение (руб)'},
    color_continuous_scale='RdYlGn_r',
    aspect='auto'
)
fig.update_layout(template='plotly_white')
fig.show()